In [1]:
import pandas as pd
import sqlite3
import os

# --- Configuration ---
DB_FILE = "movies_and_recommendations.db"
# Dictionary mapping the desired table name to its corresponding CSV file.
# The column names in the CSVs are used directly for the database tables.
CSV_FILES_TO_TABLES = {
    "genres": "genres.csv",
    "keywords": "keywords.csv",
    "movies": "movies.csv",
    "people": "people.csv"
}

# --- Database Creation and Population ---

# To ensure a fresh start, remove the old database file if it exists.
if os.path.exists(DB_FILE):
    os.remove(DB_FILE)
    print(f"Removed existing database file: '{DB_FILE}'")

try:
    # 1. Establish a connection to the SQLite database.
    # This will create the database file if it does not exist.
    conn = sqlite3.connect(DB_FILE)
    print(f"Successfully created and connected to the database: '{DB_FILE}'")

    # 2. Loop through each item in the dictionary.
    for table_name, csv_file in CSV_FILES_TO_TABLES.items():
        try:
            print(f"Reading '{csv_file}' and creating table '{table_name}'...")
            
            # Read the CSV data into a pandas DataFrame.
            df = pd.read_csv(csv_file)
            
            # Use the DataFrame's to_sql() method to write the data into the database.
            # - `name=table_name`: The name of the SQL table to be created.
            # - `con=conn`: The database connection object.
            # - `if_exists='replace'`: If the table already exists, it will be dropped and recreated.
            # - `index=False`: Prevents pandas from writing the DataFrame index as a column.
            df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)
            
            print(f"Table '{table_name}' created and populated successfully.")

        except FileNotFoundError:
            print(f"ERROR: The file '{csv_file}' was not found. Please check the file path.")
        except Exception as e:
            print(f"An error occurred while processing '{csv_file}': {e}")

    print("\n--- All tables have been created and populated. ---")

    # --- Verification Step (Optional but Recommended) ---
    print("\nVerifying the database contents...")
    cursor = conn.cursor()

    # Fetch and print the names of all tables in the database.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"Tables found in the database: {[table[0] for table in tables]}")

    # Print the first 3 rows from the 'movies' table to confirm data insertion.
    print("\nFirst 3 rows from the 'movies' table:")
    cursor.execute("SELECT id, title, release_year FROM movies LIMIT 3;")
    for row in cursor.fetchall():
        print(row)

except sqlite3.Error as e:
    print(f"DATABASE ERROR: {e}")
finally:
    # 3. Close the database connection.
    if conn:
        conn.close()
        print("\nDatabase connection closed.")

Successfully created and connected to the database: 'movies_and_recommendations.db'
Reading 'genres.csv' and creating table 'genres'...
Table 'genres' created and populated successfully.
Reading 'keywords.csv' and creating table 'keywords'...
Table 'keywords' created and populated successfully.
Reading 'movies.csv' and creating table 'movies'...
Table 'movies' created and populated successfully.
Reading 'people.csv' and creating table 'people'...
Table 'people' created and populated successfully.

--- All tables have been created and populated. ---

Verifying the database contents...
Tables found in the database: ['genres', 'keywords', 'movies', 'people']

First 3 rows from the 'movies' table:
(19995, 'Avatar', 2009)
(285, "Pirates of the Caribbean: At World's End", 2007)
(206647, 'Spectre', 2015)

Database connection closed.
